# XGBoost, aproximación primaria

## 1. Lectura de datos

### 1.1. Dataset "limpio" 

El modelo inicial de Leading Indicators, nos entrega un dataset limpio que se encarga de establecer la estructura de campañas por medio de algunos indicadores: 
* Por ejemplo, para las variables numéricas se establecen diferentes tipos de transformaciones de acuerdo con lo que se especifique en el config: 
    *  **Media ponderada**: utilizará el peso muestral (No es definido) para hallar el promedio de la variable a través de las observaciones propuestas, que para efectos del ejemplo, son semanales. 
    *  **Divisiones propuestas**: Se usa en las métricas cómo el CPM o el ROAS y son llamadas **métricas calculadas**
* Por otro lado, para las variables categóricas (aquellas contenidas dentro de la taxonomía de medios cómo el formato) se utilizan codificadores para convertirlas en dummys: 
    *  **OneHot Encoder**: Volverá una variable categórica con $k$ cateogorías en $k-1$ variables dummy dónde la primera o *base* se contará cómo un efecto natural del KPI. 
    *  **Ordenación**: Las variables categóricas se convertirán en un conteo desde $1, \dots, k$ ordenado, sin embargo esto no es recomendable ya que la ordenación podría tener interferencia sobre el resultado final del modelo y esta es completamente arbitraria. 

Leading Indicators necesita de algunas de las variables calculadas con base en otras proporcionadas con el fin de llevar a cabo métricas de negocio que deseamos que traiga directamente en el modelo. Algunos ejemplos son: 
* **Alcance de impresiones únicas**: $\frac{\sum^{n}_{i=1} \operatorname{ImpUnicas}_i \times \operatorname{Imp}_i}{\sum^{n}_{i=1}\operatorname{Imp}_i}$. El alcance de clicks únicos se calcula de la misma manera. *Variaciones* 
    * **Alcance Total**: $\frac{\sum^{n}_{i=1} \operatorname{ImpUnicas}_i \times (\operatorname{Imp}_i \times \operatorname{Clicks}_i)}{\sum^{n}_{i=1}\operatorname{Imp}_i \times \operatorname{Clicks}_i}$ 
* **CTR**: $\frac{\sum^{n}_{i=1} \operatorname{Clicks}_i}{\sum^{n}_{i=1} \operatorname{Imp}_i}$ 
* **Visibilidad / Viewability**: Existen dos métricas diferentes: 
    * **IAS**: Usando las variables de la fuente IAS, $\frac{\sum^{n}_{i=1} \operatorname{Impresiones Vistas}_i}{\sum^{n}_{i=1} \operatorname{Impresiones Medidas}_i}$
    * **DCM**: Usando las variables de la fuente DCM, $\frac{\sum^{n}_{i=1} \operatorname{Impresiones Visibles Activas}_i}{\sum^{n}_{i=1} \operatorname{Impresiones Medidas}_i}$

In [ ]:
# Escriba el código acá 

### 1.2. Eliminación de Variables Correlacionadas

Para todos los modelos con base lineal, es buena práctica eliminar todas las variables independientes que tengan una relación lineal directa o inversa con otras variables independientes, con el fin de no tener matrices singulares en el proceso de iterado.

Particularmente en LIM, se busca un cierto umbral en correlación y similaridad usando las siguientes operaciones: 
* **Correlación**: Se tiene un umbral por defecto del $|0.95|$ para todas las metodologías de correlación disponibles: 
    * Pearson, donde se asume que los dos **arrays** presentes siguen una distribución normal para que el valor reportado sea consistente y comparable. Se define cómo
    $$\rho_{(\mathsf{X,Y})} = \frac{\operatorname{Cov}(\mathsf{X,Y})}{\sqrt{\operatorname{Var}(\mathsf{X})\operatorname{Var}(\mathsf{Y})}}$$
    * Kendall, se asume que los arrays no tienen una distribución normal pero sus parejas están *adecuadamente* ordenadas (especialmente para series de tiempo) y están concordantemente o consistentemente por encima o por debajo. Se define cómo
    $$\tau_{\mathsf{X,Y}} = \frac{2}{n(n-1)}\sum_{i<j}\operatorname{sgn}(x_i - x_j)\operatorname{sgn}(y_i - y_j)$$
    * Spearman, toma los pares ordenados de los conjuntos de datos y calcula la diferencia entre los rangos. Su definición es
    $$\kappa_{\mathsf{X,Y}} = 1 - \frac{6 \sum D^2}{N(N^2-1)}$$